In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

## 配置GPU

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## 超参数

In [3]:
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [8]:
# 皮一下~用Fashion_MNIST数据集~
train_dataset = torchvision.datasets.FashionMNIST(root='../../data/fashion',
                                          train=True,
                                          transform=transforms.ToTensor(),
                                          download=True)
test_dataset = torchvision.datasets.FashionMNIST(root='../../data/fashion',
                                                train=False,
                                                transform=transforms.ToTensor())

In [11]:
# 数据生成器DataLoader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

## Convolutional neural network (two convolutional layers)

In [12]:
# 有两个卷积层的卷积神经网络
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [13]:
model = ConvNet(num_classes).to(device)

## 损失 和 优化器

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## 训练模型

In [16]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        #前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)
        #反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if(i+1)%100 == 0:
            print('Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step[100/600], Loss: 0.4434
Epoch [1/5], Step[200/600], Loss: 0.2976
Epoch [1/5], Step[300/600], Loss: 0.3106
Epoch [1/5], Step[400/600], Loss: 0.4081
Epoch [1/5], Step[500/600], Loss: 0.3521
Epoch [1/5], Step[600/600], Loss: 0.2117
Epoch [2/5], Step[100/600], Loss: 0.3241
Epoch [2/5], Step[200/600], Loss: 0.3203
Epoch [2/5], Step[300/600], Loss: 0.2124
Epoch [2/5], Step[400/600], Loss: 0.2234
Epoch [2/5], Step[500/600], Loss: 0.1273
Epoch [2/5], Step[600/600], Loss: 0.2325
Epoch [3/5], Step[100/600], Loss: 0.1918
Epoch [3/5], Step[200/600], Loss: 0.2580
Epoch [3/5], Step[300/600], Loss: 0.1905
Epoch [3/5], Step[400/600], Loss: 0.2110
Epoch [3/5], Step[500/600], Loss: 0.2038
Epoch [3/5], Step[600/600], Loss: 0.3344
Epoch [4/5], Step[100/600], Loss: 0.1478
Epoch [4/5], Step[200/600], Loss: 0.3134
Epoch [4/5], Step[300/600], Loss: 0.2849
Epoch [4/5], Step[400/600], Loss: 0.2735
Epoch [4/5], Step[500/600], Loss: 0.4289
Epoch [4/5], Step[600/600], Loss: 0.2682
Epoch [5/5], Ste

## 测试模型

In [18]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += 1
        correct = (predicted == labels).sum().item()
        
    print('Test Accuracy of the model on the 1000 test images: {}%'.format(100 * correct/total))

Test Accuracy of the model on the 1000 test images: 86.0%


## 保存模型

In [19]:
torch.save(model.state_dict(), 'model.ckpt')